In [2]:
@file:DependsOn("io.ktor:ktor-client-core:3.1.3")
@file:DependsOn("io.ktor:ktor-client-cio:3.1.3")
@file:DependsOn("io.ktor:ktor-client-content-negotiation:3.1.3")
@file:DependsOn("io.ktor:ktor-serialization-kotlinx-json:3.1.3")
@file:DependsOn("com.charleskorn.kaml:kaml-jvm:0.67.0")
@file:DependsOn("ro.jf.funds:user-sdk:1.0.0")
@file:DependsOn("ro.jf.funds:import-sdk:1.0.0")
@file:DependsOn("ro.jf.funds:fund-sdk:1.0.0")
@file:DependsOn("ro.jf.funds:account-sdk:1.0.0")
@file:DependsOn("ro.jf.funds:funds-notebook-client:1.0.0")
%use dataframe

import ro.jf.funds.client.notebook.*

val client = FundsClient()
val user = client.ensureUserExists("Johann-18.0")
user

UserTO(id=94bac0a7-88e4-4099-9a70-2ed2bf950355, username=Johann-18.0)

In [3]:
import ro.jf.funds.account.sdk.AccountSdk
import ro.jf.funds.fund.sdk.FundSdk
import ro.jf.funds.importer.sdk.ImportSdk

val ACCOUNTS_YAML_FILE = "../../data/provision/accounts.yaml"
val FUNDS_YAML_FILE = "../../data/provision/funds.yaml"
val IMPORT_CONFIGURATION_JSON_FILE = "../../data/provision/import-configuration.json"
val IMPORT_CONFIGURATION_YAML_FILE = "../../data/provision/import-configuration.yaml"

val accountSdk = AccountSdk()
val fundSdk = FundSdk()
val importSdk = ImportSdk()


In [4]:
import ro.jf.funds.account.api.model.*
import ro.jf.funds.commons.model.*
import kotlinx.coroutines.runBlocking
import com.charleskorn.kaml.Yaml
import kotlinx.serialization.builtins.ListSerializer

val accounts = runBlocking {
    val existingAccounts = accountSdk.listAccounts(user.id).items
    val existingAccountNames = existingAccounts.map { it.name }.toSet()
    val newAccounts = File(ACCOUNTS_YAML_FILE).readText()
        .let { Yaml.default.decodeFromString(ListSerializer(CreateAccountTO.serializer()), it) }
        .filter { it.name !in existingAccountNames }
        .map { accountSdk.createAccount(user.id, it) }
    existingAccounts + newAccounts
}
accounts.map { it.toString() }.joinToString(separator = "\n")

AccountTO(id=00c0f7b9-e721-4851-a952-fbf3b1a2058d, name=Cash RON, unit=Currency(value=RON))
AccountTO(id=3d7c3f1f-5c27-4119-94c5-5962f6449540, name=Cash EUR, unit=Currency(value=EUR))
AccountTO(id=74a6769b-1841-4e7a-a489-598a63863b28, name=ING RON, unit=Currency(value=RON))
AccountTO(id=0a9f02b2-2da2-47e2-b092-38a2188cf484, name=ING Economy, unit=Currency(value=RON))
AccountTO(id=40639f99-cfed-4ead-915d-89f99d69b829, name=Food Coupons, unit=Currency(value=RON))
AccountTO(id=966d0d91-19f9-4af0-b2d3-6b25abf51862, name=Revolut RON, unit=Currency(value=RON))
AccountTO(id=e36d0505-1405-421c-b9d8-58d4c9d6d3ef, name=Other, unit=Currency(value=RON))
AccountTO(id=193c3c5f-362d-4072-98c5-8142cb6e14a4, name=BT RON, unit=Currency(value=RON))
AccountTO(id=ab07c633-bc18-49b8-b662-c8493c20f6fc, name=BT Economy RON, unit=Currency(value=RON))

In [5]:
import ro.jf.funds.fund.api.model.*
import kotlinx.coroutines.runBlocking

val funds = runBlocking {
    val existingFunds = fundSdk.listFunds(user.id).items
    val existingFundNames = existingFunds.map { it.name }.toSet()
    val newFunds = File(FUNDS_YAML_FILE).readText()
        .let { Yaml.default.decodeFromString(ListSerializer(CreateFundTO.serializer()), it) }
        .filter { it.name !in existingFundNames }
        .map { fundSdk.createFund(user.id, it) }
    existingFunds + newFunds
}
funds.map { it.toString() }.joinToString(separator = "\n")

FundTO(id=3a96480a-8c4a-4ebe-b9e0-4d87bd1b9924, name=Expenses)
FundTO(id=e6a1c9b7-9db0-4fff-b951-6fa3efcaa8f9, name=Work Income)
FundTO(id=fea8ebb9-9fe7-45fa-b91a-df6705acd941, name=Savings)
FundTO(id=40925f61-e6bd-4fb5-bfb6-584da3a26137, name=Gifts)

In [6]:
import ro.jf.funds.importer.api.model.*
import kotlinx.serialization.json.Json

val dataConfigurationRawYaml: String = File(IMPORT_CONFIGURATION_YAML_FILE).readText()
val importConfiguration = Yaml.default.decodeFromString(ImportConfigurationTO.serializer(), dataConfigurationRawYaml)

In [7]:
import kotlinx.coroutines.delay
import kotlinx.coroutines.runBlocking
import kotlinx.datetime.Clock
import ro.jf.funds.importer.api.model.*
import java.io.File
import java.util.*
import kotlin.time.Duration

val csvFiles =
    listOf(File("../../data/wallet/2019"), File("../../data/wallet/2020"))
        .flatMap { it.listFiles()?.toList() ?: error("no files found") }
runBlocking {
    var importTask = importSdk.import(user.id, importConfiguration, csvFiles)
    val now: Instant = Clock.System.now()
    val timeout = 60.seconds
    while (importTask.status == ImportTaskTO.Status.IN_PROGRESS && Clock.System.now() - now < timeout) {
        delay(500)
        importTask = importSdk.getImportTask(user.id, importTask.taskId)
    }
    importTask
}


ImportTaskTO(taskId=2b02c723-5555-471f-8bd2-925eb0e09ef7, status=COMPLETED, reason=null)